In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from itertools import permutations, product
from scipy.stats import norm

from scipy.integrate import odeint
from scipy import signal 

# Initial Values

In [2]:
path = 'Dataset'
Monte = np.load(path + '/Monte.npy')

# Loading Datasets

In [3]:
lorenz_dataframe = pd.read_csv( path + '/lorenz_dataframe.csv')
ARMA_dataframe = pd.read_csv( path + '/ARMA_dataframe.csv')
WhiteNoise_dataframe = pd.read_csv( path + '/WhiteNoise_dataframe.csv')
RandomWalk_dataframe = pd.read_csv( path + '/RandomWalk_dataframe.csv')
Chirp_dataframe = pd.read_csv( path + '/Chirp_dataframe.csv')
SineWave_dataframe = pd.read_csv( path + '/SineWave_dataframe.csv')

In [4]:
time = lorenz_dataframe.to_numpy()[:,0]
lorenz_time_series =(lorenz_dataframe.to_numpy()[:,1:]).T
ARMA_time_series = (ARMA_dataframe.to_numpy()[:,1:]).T
WhiteNoise_time_series = (WhiteNoise_dataframe.to_numpy()[:,1:]).T
RandomWalk_time_series = (RandomWalk_dataframe.to_numpy()[:,1:]).T
Chirp_time_series = (Chirp_dataframe.to_numpy()[:,1:]).T
SineWave_time_series = (SineWave_dataframe.to_numpy()[:,1:]).T

# Entropies

In [5]:
def Permutation_Entropy(x, D = 3, tau = 1):
    T = len(x)
    col = T - (D-1)*tau 
    Pi = np.arange(D)
    Number_of_occ = np.zeros(np.math.factorial(D))
    for c in range(col):
        sc = tuple(np.argsort(x[c:c+D]))
        count = 0 
        for p in permutations(Pi):
            if p == sc:
                Number_of_occ[count] += 1
            count += 1   
    pi = (Number_of_occ)/col
    PE = -np.sum(pi*np.log2(pi+1e-10))
    PE_norm = 1/np.log2(np.math.factorial(D))*PE
    return PE_norm

In [6]:
def ApEn(U, m = 3, r = 0.1): 
    def _maxdist(x_i, x_j):
        return max([abs(ua - va) for ua, va in zip(x_i, x_j)])

    def _phi(m):
        x = [[U[j] for j in range(i, i + m - 1 + 1)] for i in range(N - m + 1)]
        C = [
            len([1 for x_j in x if _maxdist(x_i, x_j) <= r]) / (N - m + 1.0)
            for x_i in x
        ]
        return (N - m + 1.0) ** (-1) * sum(np.log(np.array(C)+1e-15))

    N = len(U)
    AE = _phi(m) - _phi(m + 1)
    return AE

In [7]:
def SampEn(L, m = 3, r = 0.1):
    N = len(L)
    B = 0.0
    A = 0.0

    xmi = np.array([L[i : i + m] for i in range(N - m)])
    xmj = np.array([L[i : i + m] for i in range(N - m + 1)])

    B = np.sum([np.sum(np.abs(xmii - xmj).max(axis=1) <= r) - 1 for xmii in xmi])

    m += 1
    xm = np.array([L[i : i + m] for i in range(N - m + 1)])

    A = np.sum([np.sum(np.abs(xmi - xm).max(axis=1) <= r) - 1 for xmi in xm])
    SE = -np.log(A / B)
    return SE

In [8]:
def FuzzyEn(x, m = 3, n = 3, r = 0.1):
    N = len(x)
    xmi = np.zeros((N-m, m))
    for i in range(N-m):
        xmi[i] = x[i:i+m] - np.mean(x[i:i+m])

    B = np.zeros((N-m, 1))
    for j in range(N-m):
        for i in range(N-m):
            dm = np.max(np.abs(xmi[i] - xmi[j])) 
            D = np.exp(-dm**n/r)
            B[i] += 1/(N-i-1)*np.sum(D)

    phi_m = 1/(N-m)*np.sum(B) 

    m += 1
    xm = np.zeros((N-m, m))
    for i in range(N-m):
        xm[i] = x[i:i+m] - np.mean(x[i:i+m])

    B = np.zeros((N-m, 1))
    for j in range(N-m):
        for i in range(N-m):
            dm = np.max(np.abs(xm[i] - xm[j])) 
            D = np.exp(-dm**n/r)
            B[i] += 1/(N-i-1)*np.sum(D)

    phi_m1 = 1/(N-m)*np.sum(B) 
    FE = np.log(phi_m) - np.log(phi_m1)
    return FE

In [9]:
def DistEn(x, m = 3, M = 250):
    N = len(x)
    
    xm = np.zeros((N-m, m))
    for i in range(N-m):
        xm[i] = x[i:i+m]

    D = np.zeros((N-m, N-m))
    for j in range(N-m):
        for i in range(N-m):
            D[i,j] = np.max(np.abs(xm[i] - xm[j]))

    d_vec = np.reshape(D, (-1,))
    [occ, value] = np.histogram(d_vec, bins = 10)  
    prob = occ / np.sum(occ)
    DE = -np.sum(prob*np.log(prob+1e-15)/np.log(len(prob)))
    return DE

In [10]:
def Dispersion_Entropy(x, c = 3, D = 3, tau = 1):
    y = norm.cdf(x ,loc = np.mean(x), scale = np.std(x))
    z = np.round(c*y + 0.5)
    T = len(x)
    col = T - (D-1)*tau 
    Pi = np.arange(1, c+1)
    Number_of_occ = np.zeros(c**D)
    for c_i in range(col):
        sc = tuple((z[c_i:c_i+D]))
        count = 0 
        for p in product(Pi,repeat = D):
            if p == sc:
                Number_of_occ[count] += 1
            count += 1   
    pi = (Number_of_occ)/col
    DE = -np.sum(pi*np.log(pi+1e-10))
    DE_norm = 1/np.log(c**D)*DE
    return DE_norm

In [11]:
def Entropy(time_series):
    PeEn_val = Permutation_Entropy(time_series)
    ApEn_val = ApEn(time_series)
    SampEn_val = SampEn(time_series)
    FuzzyEn_val = FuzzyEn(time_series)
    DistEn_val = DistEn(time_series)
    DispEn_val = Dispersion_Entropy(time_series)
    return [PeEn_val, ApEn_val, SampEn_val, FuzzyEn_val, DistEn_val, DispEn_val]

# Calculating Entropies

In [12]:
lorenz_entropies = np.zeros((Monte, 6))
for m in range(Monte):
    lorenz_entropies[m] = Entropy(lorenz_time_series[m])
    
df = pd.DataFrame(lorenz_entropies, columns = ['PeEn', 'ApEn', 'SampEn', 'FuzzyEn', 'DistEn', 'DispEn'])
df.to_csv(path + '/lorenz_entropies.csv', index=False)

In [13]:
ARMA_entropies = np.zeros((Monte, 6))
for m in range(Monte):
    ARMA_entropies[m] = Entropy(ARMA_time_series[m])
    
df = pd.DataFrame(ARMA_entropies, columns = ['PeEn', 'ApEn', 'SampEn', 'FuzzyEn', 'DistEn', 'DispEn'])
df.to_csv(path + '/ARMA_entropies.csv', index=False)

In [14]:
WhiteNoise_entropies = np.zeros((Monte, 6))
for m in range(Monte):
    WhiteNoise_entropies[m] = Entropy(WhiteNoise_time_series[m])
    
df = pd.DataFrame(WhiteNoise_entropies, columns = ['PeEn', 'ApEn', 'SampEn', 'FuzzyEn', 'DistEn', 'DispEn'])
df.to_csv(path + '/WhiteNoise_entropies.csv', index=False)

In [15]:
RandomWalk_entropies = np.zeros((Monte, 6))
for m in range(Monte):
    RandomWalk_entropies[m] = Entropy(RandomWalk_time_series[m])
    
df = pd.DataFrame(RandomWalk_entropies, columns = ['PeEn', 'ApEn', 'SampEn', 'FuzzyEn', 'DistEn', 'DispEn'])
df.to_csv(path + '/RandomWalk_entropies.csv', index=False)

In [16]:
Chirp_entropies = np.zeros((Monte, 6))
for m in range(Monte):
    Chirp_entropies[m] = Entropy(Chirp_time_series[m])
    
df = pd.DataFrame(Chirp_entropies, columns = ['PeEn', 'ApEn', 'SampEn', 'FuzzyEn', 'DistEn', 'DispEn'])
df.to_csv(path + '/Chirp_entropies.csv', index=False)

In [17]:
SineWave_entropies = np.zeros((Monte, 6))
for m in range(Monte):
    SineWave_entropies[m] = Entropy(SineWave_time_series[m])
    
df = pd.DataFrame(SineWave_entropies, columns = ['PeEn', 'ApEn', 'SampEn', 'FuzzyEn', 'DistEn', 'DispEn'])
df.to_csv(path + '/SineWave_entropies.csv', index=False)